# WELCOME TO CRYPTOLOGY PROJECT BY OSUNAM PANGGAM

# GETTING READY
- Please kindly upload a mp4 file for testing video and .wav file for testing Audio using the upload button [upper arrow] in left hand side. 

- once uploaded you can see your files alongside sample_data

- For ease, i also provided each of these files - .mp4 and .wav for testing.

# RUNNING NOTEBOOK

- You can run the notebook by just clicking on Runtime > Run all. These will run all cells one by one and you dont need to run for each cell.

- ** Please note on your first run - you will encounter AttributeError: module 'PIL.TiffTags' has no attribute 'IFD' for video. Hence please re-run the notebook again. because the library needs to re run once its installed. It will work fine once you re-run.**

- After you encrypt please refresh once to see the encrypted file.

#AUDIO STEGANOGRAPHY
 - The type of audio file used must be a ".wav" file because wav is one of the most popular lossless compression format.


#VIDEO STEGANOGRAPHY
 - The type of video file i used for testing is .mp4. however i tested with .mov for encryption and it seems to work fine as well. but mp4 for encryption is preferred. For decryption it already generates the .mov file.

 - Also please note that in order to view the encrypted video, the computer must have a mov player, if not you can always convert the encrypted mov video to mp4 and can see no significant change 




# INSTALLATIONS

In [ ]:
!pip install stegano 

  Using cached Pillow-8.4.0-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.1 MB)
  Attempting uninstall: pillow
    Found existing installation: Pillow 9.1.1
    Uninstalling Pillow-9.1.1:
      Successfully uninstalled Pillow-9.1.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [ ]:
!pip uninstall -y Pillow

Found existing installation: Pillow 8.4.0
Uninstalling Pillow-8.4.0:
  Successfully uninstalled Pillow-8.4.0


In [ ]:
!pip install -U pillow

  Using cached Pillow-9.1.1-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.1 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
stegano 0.9.9 requires pillow<9.0.0,>=8.2.0, but you have pillow 9.1.1 which is incompatible.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [ ]:
!sudo apt install tesseract-ocr

Reading package lists... Done
Building dependency tree       
Reading state information... Done
tesseract-ocr is already the newest version (4.00~git2288-10f4998a-2).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 42 not upgraded.


In [ ]:
!pip install pytesseract

In [ ]:
!pip install pytesseract

# VIDEO STEGANOGRAPHY - ENCRYPTION AND DECRYPTION





In [ ]:
from stegano import lsb
from os.path import isfile,join

import math
import time                                                                 
import cv2
import numpy as np
import os
import shutil
from subprocess import call,STDOUT


#function to extract frames in png format from the video file and store in a temporary folder
def extract_VideoFrames(video):
    if not os.path.exists("./temporary_folder"):
      os.makedirs("temporary_folder")
    temp_folder="./temporary_folder"
   
    vidcap = cv2.VideoCapture(video)
    count = 0

    while True:
        success, image = vidcap.read()
        if not success:
            break
        cv2.imwrite(os.path.join(temp_folder, "{:d}.png".format(count)), image)
        count += 1

#function that helps to split the secret message into letter or bytes to allocate to the frames.
def split_secret(msg,count=10):
    per_c=math.ceil(len(msg)/count)
    c_cout=0
    out_str=''
    split_list=[]
    for s in msg:
        out_str+=s
        c_cout+=1
        if c_cout == per_c:
            split_list.append(out_str)
            out_str=''
            c_cout=0
    if c_cout!=0:
        split_list.append(out_str)
    return split_list

# the least significant bit of the png frame is replaced with the secret message bit
def encrypt_text(secret_text,root="./temporary_folder/"):
    split_text_list=split_secret(secret_text)
    for i in range(0,len(split_text_list)):
        video_file="{}{}.png".format(root,i)
        secret_enc=lsb.hide(video_file,split_text_list[i])
        secret_enc.save(video_file)
        print("[INFO] frame {} holds {}".format(video_file,split_text_list[i]))

# function that helps to decrypt the video and get the secret message       
def decrypt_video(video):
    extract_VideoFrames(video)   # extract video into frames
    secret=[]
    root="./temporary_folder/"
    for i in range(len(os.listdir(root))):
        video_file="{}{}.png".format(root,i)
        secret_dec=lsb.reveal(video_file)    # gets the secret code
        if secret_dec == None:
            break
        secret.append(secret_dec)  # joins all the letter to find the message
    print('The secret message is:')
    print(''.join([i for i in secret]))
    clean_temp_file()


def clean_temp_file(path="./temporary_folder"):
    if os.path.exists(path):
        shutil.rmtree(path)
        # print("\n [INFO] Deleting temporary folder")

# function to encrpt the video 
def encrypt_video():
    secret_text = input("Enter the secret message :")
    video_file=input("enter the name of video with extention: ")
    extract_VideoFrames(video_file)                #extracts the frames from the video
    print(os.getcwd())
    # extrating audio from the video 
    call(["ffmpeg", "-i",video_file, "-q:a", "0", "-map", "a", "temporary_folder/audio.mp3", "-y"],stdout=open(os.devnull, "w"), stderr=STDOUT)
    encrypt_text(secret_text)
    call(["ffmpeg", "-i", "temporary_folder/%d.png" , "-vcodec", "png", "temporary_folder/Encrypted_video.mov", "-y"],stdout=open(os.devnull, "w"), stderr=STDOUT)
    
    #join the frames, audio together to form the encrypted video with sound
    call(["ffmpeg", "-i", "temporary_folder/Encrypted_video.mov", "-i", "temporary_folder/audio.mp3", "-codec", "copy", "Encrypted_video.mov", "-y"],stdout=open(os.devnull, "w"), stderr=STDOUT)
    clean_temp_file()
    print("Successfully encrypted video with text to Encrypted_video.mov")

#main entry point of the code
if __name__ == "__main__":
    while True:
        print("\n What would you like to do?" "\n1.Hide a secret message in video "  "\n2.Reveal the secret message from Video" "\n3.Enter to exit\n")
        choice = input()
        if choice == '1':
            encrypt_video()
        elif choice == '2':
            decrypt_video(input("Enter video name with extension"))
        else:
            break


 What would you like to do?
1.Hide a secret message in video 
2.Reveal the secret message from Video
3.Enter to exit

2
Enter video name with extensionEncrypted_video.mov
The secret message is:
Very secret message

 What would you like to do?
1.Hide a secret message in video 
2.Reveal the secret message from Video
3.Enter to exit



# AUDIO STEGANOGRAPHY - ENCRYPTION AND DECRYPTION

In [ ]:
import wave   # this library helps to deal with the audio file .wav

def encrypt_audio(audio, secret):   #function to encrypt the audio with text
	print("\n Encoding Audio..")
	audio = wave.open(audio,mode="rb")
	frame_bytes = bytearray(list(audio.readframes(audio.getnframes())))   #divides the audio into frames - bytes
	# secret = str(secret_message)
	# print(secret)
	secret = secret + int((len(frame_bytes)-(len(secret)*8*8))/8) *'#'
	bits = list(map(int, ''.join([bin(ord(i)).lstrip('0b').rjust(8,'0') for i in secret])))  #maps the audio byte with the secret text bit in lsb
	for i, bit in enumerate(bits):                                            
	    frame_bytes[i] = (frame_bytes[i] & 254) | bit
	modify_frame = bytes(frame_bytes)        #
	# for i in range(0,10):
	# 	print(frame_bytes[i])
	EncryptedAudio =  wave.open('Encrypted_Audio.wav', 'wb')
	EncryptedAudio.setparams(audio.getparams())
	EncryptedAudio.writeframes(modify_frame)

	EncryptedAudio.close()
	audio.close()
	print("Successfully encrypted audio with text to Encrypted_Audio.wav")

def decrypt_audio(audio):                #function to decrypt the audio and get the secret message
	print("\n Decoding Audio..")
	audio = wave.open(audio, mode='rb')
	frame_bytes = bytearray(list(audio.readframes(audio.getnframes())))         # gets a list which has frames extracted from the audio
	extracted_audio = [frame_bytes[i] & 1 for i in range(len(frame_bytes))]
	secret = "".join(chr(int("".join(map(str,extracted_audio[i:i+8])),2)) for i in range(0,len(extracted_audio),8))  #joins all the letters of the text
	decoded = secret.split("###")[0]
	print("Sucessfully decoded - The secret Message is : "+decoded)
	audio.close()	
 
def call_encrypt_audio_function():  # an additional function i made to take input from user.
    secret_message = input("Enter the secret message :")
    audio_file=input("Enter the name of Audio file to encrypt: ")
    encrypt_audio(audio_file, secret_message)


#this is the main function - entry point.
if __name__ == "__main__":
    while True:
        print("What would you like to do?" "\n1.Hide a message in Audio"  "\n2.Reveal the secret from Audio" "\n3.Enter to exit")
        choice = input()
        if choice == '1':
            call_encrypt_audio_function()
        elif choice == '2':
            decrypt_audio(input("enter the name of audio with extension"))
        else:
            break

What would you like to do?
1.Hide a message in Audio
2.Reveal the secret from Audio
3.Enter to exit

